In [ ]:
!pip install holidays

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv')

In [ ]:
train_min_date = min(train['date'])
train_max_date = max(train['date'])
test_min_date = min(test['date'])
test_max_date = max(test['date'])

df = pd.concat([train, test])
df

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.weekday
df['is_weekend'] = np.where((df['weekday']==5) | (df['weekday']==6), 1, 0)

In [ ]:
import holidays


holiday_FI = holidays.CountryHoliday('FI', years=[2015, 2016, 2017, 2018, 2019])
holiday_NO = holidays.CountryHoliday('NO', years=[2015, 2016, 2017, 2018, 2019])
holiday_SE = holidays.CountryHoliday('SE', years=[2015, 2016, 2017, 2018, 2019])

holiday_dict = holiday_FI.copy()
holiday_dict.update(holiday_NO)
holiday_dict.update(holiday_SE)

df['holiday_name'] = df['date'].map(holiday_dict)
df['is_holiday'] = np.where(df['holiday_name'].notnull(), 1, 0)
df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')
df = pd.get_dummies(df, columns=['country', 'store', 'product', 'holiday_name'])

In [ ]:
df.head()

In [ ]:
# trainとtestに分割
train_df = df[(df['date']>=train_min_date) & (df['date']<=train_max_date)].copy()
test_df = df[(df['date']>=test_min_date) & (df['date']<=test_max_date)].copy()
train_df

In [ ]:
train_df.drop(['row_id', 'date'], axis=1, inplace=True)
train_df.head()

In [ ]:
df_1 = train_df.drop('num_sold', axis=1)

In [ ]:
df_1.head()

In [ ]:
x = df_1.copy()

In [ ]:
test_df.drop(['date', 'row_id', 'num_sold'], axis=1, inplace=True)
test_df.head()

In [ ]:
len(df_1.columns), len(test_df.columns)

In [ ]:
y = train_df['num_sold']
y.head()

In [ ]:
from catboost import CatBoostRegressor


model = CatBoostRegressor()
model.fit(x, y)

In [ ]:
prediction = model.predict(test_df)
prediction

In [ ]:
submission['num_sold'] = prediction
submission.to_csv('submission.csv', index=False)
submission.head()